In [2]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import k3d
import numpy as np
from bmcs_shell.folding.assembly.wb_scanned_cell import WBScannedCell
from bmcs_shell.api import WBTessellation4PEx
from scipy.spatial import cKDTree
import pandas as pd
from openpyxl import load_workbook
import sympy as sp
from IPython.display import display
from sympy import Eq, Symbol

In [3]:


# Set the reference geometry to be generated from the closed form kinematics
t_num = 15
a_num= 1000/4
b_num= 1615/4
c_num= 645/4
e_num= 286/4
gamma_num= 0.683


wb_shell = WBTessellation4PEx(
                         a = a_num,
                         b = b_num, 
                         c = c_num, 
                         e_x = e_num,
                         gamma=gamma_num, # a value of gamma = 0.75 was estimated from normals, but a CAD comparison showed that 0.75 doesn't lead to closer geometry to the scanned 
                         n_phi_plus=2, # planned 5 
                         n_x_plus=2,  # planned 3
                         wireframe_width=5,
                        ##---- Trimming function works only in WBTessellation4P ----##
                         trim_half_cells_along_y=True,
                         trim_half_cells_along_x=True,
#                          align_outer_nodes_along_x=True,
)
# wb_shell.interact()
orig_I_Fi = np.copy(wb_shell.I_Fi_trimmed)
orig_X_Ia = np.copy(wb_shell.X_Ia_trimmed)

Derivation of the crease nodes of the lower facet (-)

![Crease convention and nomenclature](wb_nomenclature_analytical.png)

In [ ]:
# Define symbols
a, b, c, e, gamma, t = sp.symbols(r'a b c e \gamma t', real=True)

alpha = sp.pi/2 - gamma


In [5]:
# Substitute parameters
subs_dict = {
    a: a_num,
    b: b_num,
    c: c_num,
    e: e_num,
    gamma: gamma_num,
    t: t_num,
}

subs_dict

{a: 250.0, b: 403.75, c: 161.25, e: 71.5, \gamma: 0.683, t: 15}

In [6]:
# Known angles (alpha_x and alpha_z)
theta_x = sp.Symbol('theta_x', real=True)
theta_y = sp.Symbol('theta_y', real=True)  # Unknown
theta_z = sp.Symbol('theta_z', real=True)

# Trig functions
cx = sp.cos(theta_x)
sx = sp.sin(theta_x)
cy = sp.cos(theta_y)
sy = sp.sin(theta_y)
cz = sp.cos(theta_z)
sz = sp.sin(theta_z)

# Rotation matrices
R_x = sp.Matrix([
    [1, 0, 0],
    [0, cx, -sx],
    [0, sx, cx]
])
R_y = sp.Matrix([
    [cy, 0, sy],
    [0, 1, 0],
    [-sy, 0, cy]
])
R_z = sp.Matrix([
    [cz, -sz, 0],
    [sz, cz, 0],
    [0, 0, 1]
])

# Full rotation matrix (Z * Y * X)
#R = R_z @ R_y @ R_x
R_x


Matrix([
[1,            0,             0],
[0, cos(theta_x), -sin(theta_x)],
[0, sin(theta_x),  cos(theta_x)]])

In [7]:
x_folded = a+e
display(Eq(sp.Symbol(r'x^{\urcorner}'), x_folded))

Eq(x^{\urcorner}, a + e)

In [8]:
y_folded = sp.sqrt((b**2*sp.cos(gamma)**2) - a**2 * (1 - sp.sin(gamma))**2) / (sp.cos(gamma))
display(Eq(sp.Symbol(r'y^{\urcorner}'), y_folded))

Eq(y^{\urcorner}, sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))

In [9]:
z_folded = (a * (1 - sp.sin(gamma))) / sp.cos(gamma)
display(Eq(sp.Symbol(r'z^{\urcorner}'), z_folded))

Eq(z^{\urcorner}, a*(1 - sin(\gamma))/cos(\gamma))

In [10]:
# Calculating theta to follow the curvature of the tesselation
theta = ((z_folded - c*sp.cos(gamma))**2- y_folded**2) / ((z_folded - c*sp.cos(gamma))**2+ y_folded**2)
theta = -sp.pi +sp.acos(theta.simplify())
display(Eq(Symbol(r'\theta_x'), theta, evaluate=False)) 

Eq(\theta_x, acos((-a**2*(sin(\gamma) - 1)**2 + b**2*cos(\gamma)**2 - (a*(sin(\gamma) - 1) + c*cos(\gamma)**2)**2)/((-2*a*c*sin(\gamma) + 2*a*c - b**2 - c**2*cos(\gamma)**2)*cos(\gamma)**2)) - pi)

In [11]:
theta_x_num = np.array(theta.subs(subs_dict)).astype(np.float64).flatten().tolist()[0]
theta_x_num

-0.03208748520889472

In [12]:
# Substitute parameters
subs_dict = {
    a: a_num,
    b: b_num,
    c: c_num,
    e: e_num,
    gamma: gamma_num,
    t: t_num,
    theta_x: theta_x_num,
}

subs_dict

{a: 250.0,
 b: 403.75,
 c: 161.25,
 e: 71.5,
 \gamma: 0.683,
 t: 15,
 theta_x: -0.03208748520889472}

In [13]:
O_gamma_center_lower = sp.Matrix([0, 0, 0])
display(Eq(Symbol(r'O_{\gamma-}^{+}'), O_gamma_center_lower, evaluate=False))

Eq(O_{\gamma-}^{+}, Matrix([
[0],
[0],
[0]]))

In [14]:
C_gamma_mr_lower = sp.Matrix([e, 0, 0])
C_gamma_mr_lower
display(Eq(Symbol(r'C_{\gamma-}^{\dashv}'), C_gamma_mr_lower, evaluate=False))

Eq(C_{\gamma-}^{\dashv}, Matrix([
[e],
[0],
[0]]))

In [15]:
C_gamma_mr_lower_num = np.array(C_gamma_mr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
C_gamma_mr_lower_num, orig_X_Ia[0]

([71.5, 0.0, 0.0], array([71.5,  0. ,  0. ]))

In [16]:
C_gamma_ml_lower = sp.Matrix([-e, 0, 0])
C_gamma_ml_lower
display(Eq(Symbol(r'C_{\gamma-}^{\vdash}'), C_gamma_ml_lower, evaluate=False))

Eq(C_{\gamma-}^{\vdash}, Matrix([
[-e],
[ 0],
[ 0]]))

In [17]:
C_gamma_ml_lower_num = np.array(C_gamma_ml_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
C_gamma_ml_lower_num, orig_X_Ia[1]

([-71.5, 0.0, 0.0], array([-71.5,   0. ,   0. ]))

In [18]:
V_gamma_mr_lower = sp.Matrix([e+c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'V_{\gamma-}^{\dashv}'), V_gamma_mr_lower, evaluate=False))

Eq(V_{\gamma-}^{\dashv}, Matrix([
[c*sin(\gamma) + e],
[                0],
[    c*cos(\gamma)]]))

In [19]:
V_gamma_mr_lower_num = np.array(V_gamma_mr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
V_gamma_mr_lower_num, orig_X_Ia[6]

([173.26856909385202, 0.0, 125.07885850370505],
 array([173.26856995,   0.        , 125.07910156]))

In [20]:
V_gamma_ml_lower = sp.Matrix([-e-c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'V_{\gamma-}^{\dashv}'), V_gamma_ml_lower, evaluate=False))

Eq(V_{\gamma-}^{\dashv}, Matrix([
[-c*sin(\gamma) - e],
[                 0],
[     c*cos(\gamma)]]))

In [21]:
V_gamma_ml_lower_num = np.array(V_gamma_ml_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
V_gamma_ml_lower_num, orig_X_Ia[7]

([-173.26856909385202, 0.0, 125.07885850370505],
 array([-173.26856995,    0.        ,  125.07910156]))

In [22]:
U_gamma_ur_lower = sp.Matrix([x_folded, y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\urcorner}'), U_gamma_ur_lower, evaluate=False))

Eq(U_{\gamma-}^{\urcorner}, Matrix([
[                                                             a + e],
[sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                   a*(1 - sin(\gamma))/cos(\gamma)]]))

In [23]:
U_gamma_ur_lower_num = np.array(U_gamma_ur_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_gamma_ur_lower_num, orig_X_Ia[2]

([321.5, 385.8493740048242, 118.88785926277468],
 array([321.5       , 385.84936523, 118.88867188]))

In [24]:
U_gamma_ul_lower = sp.Matrix([-x_folded, y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\ulcorner}'), U_gamma_ur_lower, evaluate=False))

Eq(U_{\gamma-}^{\ulcorner}, Matrix([
[                                                             a + e],
[sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                   a*(1 - sin(\gamma))/cos(\gamma)]]))

In [25]:
U_gamma_ul_lower_num = np.array(U_gamma_ul_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_gamma_ul_lower_num, orig_X_Ia[3]

([-321.5, 385.8493740048242, 118.88785926277468],
 array([-321.5       ,  385.84936523,  118.88867188]))

In [26]:
U_gamma_ll_lower = sp.Matrix([-x_folded, -y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\llcorner}'), U_gamma_ll_lower, evaluate=False))

Eq(U_{\gamma-}^{\llcorner}, Matrix([
[                                                             -a - e],
[-sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                    a*(1 - sin(\gamma))/cos(\gamma)]]))

In [27]:
U_gamma_ll_lower_num = np.array(U_gamma_ll_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_gamma_ll_lower_num, orig_X_Ia[5]

([-321.5, -385.8493740048242, 118.88785926277468],
 array([-321.5       , -385.84936523,  118.88867188]))

In [28]:
U_gamma_lr_lower = sp.Matrix([x_folded, -y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\llcorner}'), U_gamma_ll_lower, evaluate=False))

Eq(U_{\gamma-}^{\llcorner}, Matrix([
[                                                             -a - e],
[-sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                    a*(1 - sin(\gamma))/cos(\gamma)]]))

In [29]:
U_gamma_lr_lower_num = np.array(U_gamma_lr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_gamma_lr_lower_num, orig_X_Ia[4]

([321.5, -385.8493740048242, 118.88785926277468],
 array([ 321.5       , -385.84936523,  118.88867188]))

In [30]:
delta_theta_ur = V_gamma_mr_lower - R_x * U_gamma_ll_lower
display(Eq(Symbol(r'\delta_{\theta}^{\urcorner}'), delta_theta_ur, evaluate=False))

Eq(\delta_{\theta}^{\urcorner}, Matrix([
[                                                                                                                        a + c*sin(\gamma) + 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [31]:
delta_theta_ul = V_gamma_ml_lower - R_x * U_gamma_lr_lower
display(Eq(Symbol(r'\delta_{\theta}^{\ulcorner}'), delta_theta_ul, evaluate=False))

Eq(\delta_{\theta}^{\ulcorner}, Matrix([
[                                                                                                                       -a - c*sin(\gamma) - 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [32]:
M_gamma_ur_lower = R_x @ -C_gamma_mr_lower + delta_theta_ur
display(Eq(Symbol(r'M_{\gamma-}^{\urcorner}'), M_gamma_ur_lower, evaluate=False))

Eq(M_{\gamma-}^{\urcorner}, Matrix([
[                                                                                                                          a + c*sin(\gamma) + e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [33]:
M_gamma_ur_lower_num = np.array(M_gamma_ur_lower.subs(subs_dict).evalf(), dtype=float).flatten()
M_gamma_ur_lower_num, orig_X_Ia[14]

(array([423.26856909, 381.83659666,  -6.12661375]),
 array([423.26856909, 381.83660889,  -6.12597656]))

In [34]:
M_gamma_ul_lower = M_gamma_ur_lower.multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'M_{\gamma-}^{\ulcorner}'), M_gamma_ul_lower, evaluate=False))

Eq(M_{\gamma-}^{\ulcorner}, Matrix([
[                                                                                                                         -a - c*sin(\gamma) - e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [35]:
M_gamma_ul_lower_num = np.array(M_gamma_ul_lower.subs(subs_dict).evalf(), dtype=float).flatten()
M_gamma_ul_lower_num, orig_X_Ia[8]

(array([-423.26856909,  381.83659666,   -6.12661375]),
 array([-423.26856909,  381.83660889,   -6.12597656]))

In [36]:
M_gamma_ll_lower = M_gamma_ur_lower.multiply_elementwise(sp.Matrix([-1, -1, 1]))
display(Eq(Symbol(r'M_{\gamma-}^{\llcorner}'), M_gamma_ll_lower, evaluate=False))

Eq(M_{\gamma-}^{\llcorner}, Matrix([
[                                                                                                                         -a - c*sin(\gamma) - e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [37]:
M_gamma_ll_lower_num = np.array(M_gamma_ll_lower.subs(subs_dict).evalf(), dtype=float).flatten()
M_gamma_ll_lower_num, orig_X_Ia[11]

(array([-423.26856909, -381.83659666,   -6.12661375]),
 array([-423.26856909, -381.83660889,   -6.12597656]))

In [38]:
M_gamma_lr_lower = M_gamma_ur_lower.multiply_elementwise(sp.Matrix([1, -1, 1]))
display(Eq(Symbol(r'M_{\gamma-}^{\lrcorner}'), M_gamma_lr_lower, evaluate=False))

Eq(M_{\gamma-}^{\lrcorner}, Matrix([
[                                                                                                                          a + c*sin(\gamma) + e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [39]:
M_gamma_lr_lower_num = np.array(M_gamma_lr_lower.subs(subs_dict).evalf(), dtype=float).flatten()
M_gamma_lr_lower_num, orig_X_Ia[17]

(array([ 423.26856909, -381.83659666,   -6.12661375]),
 array([ 423.26856909, -381.83660889,   -6.12597656]))

In [40]:
H_gamma_ur_lower = delta_theta_ur
display(Eq(Symbol(r'H_{\gamma-}^{\urcorner}'), H_gamma_ur_lower, evaluate=False))

Eq(H_{\gamma-}^{\urcorner}, Matrix([
[                                                                                                                        a + c*sin(\gamma) + 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [41]:
H_gamma_ur_lower_num = np.array(H_gamma_ur_lower.subs(subs_dict).evalf(), dtype=float).flatten()
H_gamma_ur_lower_num, orig_X_Ia[13]

(array([494.76856909, 381.83659666,  -6.12661375]),
 array([494.76856909, 381.83660889,  -6.12597656]))

In [42]:
H_gamma_ul_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'H_{\gamma-}^{\ulcorner}'), H_gamma_ul_lower, evaluate=False))

Eq(H_{\gamma-}^{\ulcorner}, Matrix([
[                                                                                                                       -a - c*sin(\gamma) - 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [43]:
H_gamma_ul_lower_num = np.array(H_gamma_ul_lower.subs(subs_dict).evalf(), dtype=float).flatten()
H_gamma_ul_lower_num, orig_X_Ia[9]

(array([-494.76856909,  381.83659666,   -6.12661375]),
 array([-494.76856909,  381.83660889,   -6.12597656]))

In [44]:
H_gamma_ll_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([-1, -1, 1]))
display(Eq(Symbol(r'H_{\gamma-}^{\llcorner}'), H_gamma_ll_lower, evaluate=False))

Eq(H_{\gamma-}^{\llcorner}, Matrix([
[                                                                                                                       -a - c*sin(\gamma) - 2*e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [45]:
H_gamma_ll_lower_num = np.array(H_gamma_ll_lower.subs(subs_dict).evalf(), dtype=float).flatten()
H_gamma_ll_lower_num, orig_X_Ia[12]

(array([-494.76856909, -381.83659666,   -6.12661375]),
 array([-494.76856909, -381.83660889,   -6.12597656]))

In [46]:
H_gamma_lr_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([1, -1, 1]))
display(Eq(Symbol(r'H_{\gamma-}^{\llcorner}'), H_gamma_lr_lower, evaluate=False))

Eq(H_{\gamma-}^{\llcorner}, Matrix([
[                                                                                                                        a + c*sin(\gamma) + 2*e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [47]:
H_gamma_lr_lower_num = np.array(H_gamma_lr_lower.subs(subs_dict).evalf(), dtype=float).flatten()
H_gamma_lr_lower_num, orig_X_Ia[16]

(array([ 494.76856909, -381.83659666,   -6.12661375]),
 array([ 494.76856909, -381.83660889,   -6.12597656]))

In [48]:
H_gamma_mr_lower = sp.Matrix([a + 2*e+c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'H_{\gamma-}^{\dashv}'), H_gamma_mr_lower, evaluate=False))

Eq(H_{\gamma-}^{\dashv}, Matrix([
[a + c*sin(\gamma) + 2*e],
[                      0],
[          c*cos(\gamma)]]))

In [49]:
H_gamma_mr_lower_num = np.array(H_gamma_mr_lower.subs(subs_dict)).astype(np.float64).flatten()
H_gamma_mr_lower_num, orig_X_Ia[15]

(array([494.76856909,   0.        , 125.0788585 ]),
 array([ 4.94768554e+02, -7.62939453e-06,  1.25079102e+02]))

In [50]:
H_gamma_ml_lower = sp.Matrix([-a-2*e-c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'H_{\gamma-}^{\vdash}'), H_gamma_ml_lower, evaluate=False))

Eq(H_{\gamma-}^{\vdash}, Matrix([
[-a - c*sin(\gamma) - 2*e],
[                       0],
[           c*cos(\gamma)]]))

In [51]:
H_gamma_ml_lower_num = np.array(H_gamma_ml_lower.subs(subs_dict)).astype(np.float64).flatten()
H_gamma_ml_lower_num, orig_X_Ia[10]

(array([-494.76856909,    0.        ,  125.0788585 ]),
 array([-4.94768554e+02, -7.62939453e-06,  1.25079102e+02]))

In [52]:
O_gamma_center_upper = sp.Matrix([0, 0, t])
display(Eq(Symbol(r'O_{\gamma+}^{+}'), O_gamma_center_upper, evaluate=False))

Eq(O_{\gamma+}^{+}, Matrix([
[0],
[0],
[t]]))

In [53]:
C_gamma_mr_upper = sp.Matrix([e-t*sp.tan(alpha/2), 0, t])
display(Eq(Symbol(r'C_{\gamma+}^{\dashv}'), C_gamma_mr_upper, evaluate=False))


Eq(C_{\gamma+}^{\dashv}, Matrix([
[e - t*cot(\gamma/2 + pi/4)],
[                         0],
[                         t]]))

In [54]:
V_gamma_mr_upper = sp.Matrix([e+c*sp.sin(gamma)-t*sp.tan(alpha/2), 0, c*sp.cos(gamma)+t])
display(Eq(Symbol(r'V_{\gamma+}^{\dashv}'), V_gamma_mr_upper, evaluate=False))


Eq(V_{\gamma+}^{\dashv}, Matrix([
[c*sin(\gamma) + e - t*cot(\gamma/2 + pi/4)],
[                                         0],
[                         c*cos(\gamma) + t]]))

In [55]:
H_gamma_mr_upper = sp.Matrix([a+2*e+c*sp.sin(gamma),0,  c*sp.cos(gamma)+t])
display(Eq(Symbol(r'H_{\gamma+}^{\dashv}'), H_gamma_mr_upper, evaluate=False))


Eq(H_{\gamma+}^{\dashv}, Matrix([
[a + c*sin(\gamma) + 2*e],
[                      0],
[      c*cos(\gamma) + t]]))

In [56]:
C_gamma_ml_upper = sp.Matrix([e-t*sp.tan(alpha/2), 0, t]).multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'C_{\gamma+}^{\vdash}'), C_gamma_ml_upper, evaluate=False))

Eq(C_{\gamma+}^{\vdash}, Matrix([
[-e + t*cot(\gamma/2 + pi/4)],
[                          0],
[                          t]]))

In [57]:
V_gamma_ml_upper = sp.Matrix([e+c*sp.sin(gamma)-t*sp.tan(alpha/2), 0, c*sp.cos(gamma)+t]).multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'V_{\gamma+}^{\vdash}'), V_gamma_ml_upper, evaluate=False))

Eq(V_{\gamma+}^{\vdash}, Matrix([
[-c*sin(\gamma) - e + t*cot(\gamma/2 + pi/4)],
[                                          0],
[                          c*cos(\gamma) + t]]))

In [58]:
H_gamma_ml_upper = sp.Matrix([a+2*e+c*sp.sin(gamma),0,  c*sp.cos(gamma)+t]).multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'H_{\gamma+}^{\vdash}'), H_gamma_ml_upper, evaluate=False))


Eq(H_{\gamma+}^{\vdash}, Matrix([
[-a - c*sin(\gamma) - 2*e],
[                       0],
[       c*cos(\gamma) + t]]))

In [59]:
Lx = 2 * (a + c + e)
Ly = 2*b

In [62]:
area = Lx * Ly
area

2*b*(2*a + 2*c + 2*e)

In [78]:
vol_tot = area * t
vol_tot

2*b*t*(2*a + 2*c + 2*e)

In [73]:
s_x, s_y, rho = sp.symbols(r's_x s_y \rho', real=True)

n_x = Lx / s_x
n_y = Ly / s_y

In [74]:
n_x

(2*a + 2*c + 2*e)/s_x

In [75]:
n_x.free_symbols

{a, c, e, s_x}

In [80]:
mass = vol_tot * rho
mass

2*\rho*b*t*(2*a + 2*c + 2*e)

In [81]:
mass.latexpr(s_x, s_y, rho)

AttributeError: 'Mul' object has no attribute 'latexpr'